In [ ]:
# Ignore SQLITE warnings related to Decimal numbers in the Chinook database
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func


In [3]:
# Create an engine for the chinook.sqlite database
engine = create_engine("sqlite:///../Resources/chinook.sqlite", echo=False)

In [4]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'tracks',
 'media_types',
 'playlists']

In [5]:
# Save a reference to the invoices table as `Invoices`
Invoices = Base.classes.invoices

In [6]:
# Create a database session object
session = Session(engine)

In [9]:
# List all of the countries found in the invoices table
countries = session.query(Invoices.BillingCountry).distinct(Invoices.BillingCountry).all()
countries


[('Germany'),
 ('Norway'),
 ('Belgium'),
 ('Canada'),
 ('USA'),
 ('France'),
 ('Ireland'),
 ('United Kingdom'),
 ('Australia'),
 ('Chile'),
 ('India'),
 ('Brazil'),
 ('Portugal'),
 ('Netherlands'),
 ('Spain'),
 ('Sweden'),
 ('Czech Republic'),
 ('Finland'),
 ('Denmark'),
 ('Italy'),
 ('Poland'),
 ('Austria'),
 ('Hungary'),
 ('Argentina')]

In [15]:
# Design a query that lists the invoices totals for each billing country 
# and sort the output in descending order.
bill_countries_total = session.query(Invoices.BillingCountry,func.sum(Invoices.Total)).\
group_by(Invoices.BillingCountry).\
order_by(func.sum(Invoices.Total).desc()).\
all()
bill_countries_total

[('USA', Decimal('523.06')),
 ('Canada', Decimal('303.96')),
 ('France', Decimal('195.10')),
 ('Brazil', Decimal('190.10')),
 ('Germany', Decimal('156.48')),
 ('United Kingdom', Decimal('112.86')),
 ('Czech Republic', Decimal('90.24')),
 ('Portugal', Decimal('77.24')),
 ('India', Decimal('75.26')),
 ('Chile', Decimal('46.62')),
 ('Ireland', Decimal('45.62')),
 ('Hungary', Decimal('45.62')),
 ('Austria', Decimal('42.62')),
 ('Finland', Decimal('41.62')),
 ('Netherlands', Decimal('40.62')),
 ('Norway', Decimal('39.62')),
 ('Sweden', Decimal('38.62')),
 ('Poland', Decimal('37.62')),
 ('Italy', Decimal('37.62')),
 ('Denmark', Decimal('37.62')),
 ('Australia', Decimal('37.62')),
 ('Argentina', Decimal('37.62')),
 ('Spain', Decimal('37.62')),
 ('Belgium', Decimal('37.62'))]

In [16]:
# Save a reference to the invoice_items table as `Items`
Items = Base.classes.invoice_items

In [18]:
# List all of the Billing Postal Codes for the USA.
Bill_postal = session.query(Invoices.BillingPostalCode).filter(Invoices.BillingCountry=='USA').distinct().all()
Bill_postal

[('2113'),
 ('94043-1351'),
 ('98052-8300'),
 ('95014'),
 ('89503'),
 ('53703'),
 ('85719'),
 ('76110'),
 ('84102'),
 ('32801'),
 ('60611'),
 ('10012-2612'),
 ('94040-111')]

In [23]:
# Calculate the Item Totals (sum(UnitPrice * Quantity)) for the USA
session.query(func.sum(Items.UnitPrice * Items.Quantity)).filter(Invoices.InvoiceId ==Items.InvoiceId).filter(Invoices.BillingCountry=='USA').all()

C:\Users\mail2\anaconda3\lib\site-packages\sqlalchemy\sql\sqltypes.py:661: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


[(Decimal('523.0600000000'))]

In [26]:
# Calculate the Item Totals `sum(UnitPrice * Quantity)` for each Billing Postal Code in the USA
# Sort the results in descending order by Total
session.query(Invoices.BillingPostalCode,func.sum(Items.UnitPrice * Items.Quantity)).\
filter(Invoices.InvoiceId ==Items.InvoiceId).\
filter(Invoices.BillingCountry=='USA').\
group_by(Invoices.BillingPostalCode).\
order_by(func.sum(Items.UnitPrice * Items.Quantity).desc()).\
all()

[('76110', Decimal('47.6200000000')),
 ('60611', Decimal('43.6200000000')),
 ('84102', Decimal('43.6200000000')),
 ('53703', Decimal('42.6200000000')),
 ('94040-111', Decimal('39.6200000000')),
 ('98052-8300', Decimal('39.6200000000')),
 ('32801', Decimal('39.6200000000')),
 ('95014', Decimal('38.6200000000')),
 ('94043-1351', Decimal('37.6200000000')),
 ('89503', Decimal('37.6200000000')),
 ('85719', Decimal('37.6200000000')),
 ('2113', Decimal('37.6200000000')),
 ('10012-2612', Decimal('37.6200000000'))]